In [3]:
# Using scarches kernel
import sys
sys.path.append("../../function/")
import trajDiff_v129
milo2=trajDiff_v129.Milo2()
import scanpy as sc
import numpy as np
import pandas as pd
import PyComplexHeatmap
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'trajDiff_v129'

In [2]:
dpt=sc.read("../../important_processed_data/11.16_dpt.h5ad")

In [3]:
timeDf=pd.read_csv("../../important_processed_data/11.19_lightGBM_pred_and_orig.csv",index_col=0)

In [4]:
dpt.obs["light_pred"]=timeDf["light_pred"]

In [7]:
dpt.layers["logCount"]=dpt.X

In [8]:
dpt.X=dpt.layers["counts"]

In [10]:
del dpt.obsm["X_diffmap"]

In [12]:
dpt_mes=dpt[dpt.obs["lineage_mesenchyme"]]
mdata_mes = milo2.load(dpt_mes)

In [40]:
def runCPM(mdata):
    sc.pp.neighbors(mdata['rna'], use_rep='X_scANVI', n_neighbors=200, n_pcs=15)
    milo2.make_nhoods(mdata['rna'], prop=0.015)
    mdata = milo2.count_nhoods(mdata['rna'], sample_col="Sample")
    pseudobulk=milo2.make_pseudobulk_parallel(mdata=mdata,sample_col="Sample",group_col="Stage",time_col="light_pred",njob=25,min_cell=2)
    milo2.make_range(mdata,only_range=True)
    wholeCpm=milo2.make_whole_cpm(mdata)
    milo2.permute_point_cpm_parallel(mdata,n_jobs=25)
    return(mdata)

In [29]:
mdata=mdata_mes
sc.pp.neighbors(mdata['rna'], use_rep='X_scANVI', n_neighbors=200, n_pcs=15)
milo2.make_nhoods(mdata['rna'], prop=0.015)
mdata = milo2.count_nhoods(mdata['rna'], sample_col="Sample")
pseudobulk=milo2.make_pseudobulk_parallel(mdata=mdata,sample_col="Sample",group_col="Stage",time_col="light_pred",njob=25,min_cell=2)
milo2.make_range(mdata,only_range=True)
wholeCpm=milo2.make_whole_cpm(mdata)
milo2.permute_point_cpm_parallel(mdata)

/home/zhanglab/mambaforge/envs/celloracle/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
100%|██████████| 879/879 [09:55<00:00,  1.48it/s]


Using edgeR to find CPM......

  0%|          | 1/879 [00:22<5:24:28, 22.17s/it]Unable to determine R library path: Command '('/usr/lib/R/bin/Rscript', '-e', 'cat(Sys.getenv("LD_LIBRARY_PATH"))')' returned non-zero exit status 2.
Unable to determine R library path: Command '('/usr/lib/R/bin/Rscript', '-e', 'cat(Sys.getenv("LD_LIBRARY_PATH"))')' returned non-zero exit status 2.
Unable to determine R library path: Command '('/usr/lib/R/bin/Rscript', '-e', 'cat(Sys.getenv("LD_LIBRARY_PATH"))')' returned non-zero exit status 2.
Unable to determine R library path: Command '('/usr/lib/R/bin/Rscript', '-e', 'cat(Sys.getenv("LD_LIBRARY_PATH"))')' returned non-zero exit status 2.
Unable to determine R library path: Command '('/usr/lib/R/bin/Rscript', '-e', 'cat(Sys.getenv("LD_LIBRARY_PATH"))')' returned non-zero exit status 2.
Unable to determine R library path: Command '('/usr/lib/R/bin/Rscript', '-e', 'cat(Sys.getenv("LD_LIBRARY_PATH"))')' returned non-zero exit status 2.
Unable to determine R library path: Command '('/usr

In [43]:
mdata["milo"].uns["cpm_dict"].keys()

dict_keys(['CranioSoxc_Angelozzi_WTE17.5', 'Forelimb_He_E11', 'LimbG610C_Gorrell_femurWT4', 'Forelimb_He_E15_2', 'LimbMouse2019_Kelly_E13', 'Maxillary_Bian_E11.5', 'Maxillary_Bian_E12.5', 'Mandible2020_Chai', 'Forelimb_He_E13.5', 'LimbG610C_Gorrell_femurWT1', 'LimbG610C_Gorrell_femurWT3', 'Mandible2020_Chai_E12', 'Forelimb_He_E15_1', 'Frontal_Holmes_WTE18.5_2', 'Maxillary_Bian_E10.5', 'LimbMouse2019_Kelly_E15', 'coronal_Holmes_P10', 'Mesenchymal2022_Zhang_E14.5', 'Maxillary_Bian_E14.5', 'Frontal_Holmes_WTP28', 'CranioSoxc_Angelozzi_WTE12.5', 'Suture2021_Farmer_E17', 'coronal_Holmes_E18_2', 'Mesenchymal2022_Zhang_E10', 'CranioSoxc_Angelozzi_WTE15.5', 'coronal_Holmes_E18_1', 'lambdoid_Holmes_P28', 'PerichondrialP21_Matsushita_Prrx1creE11.5', 'Forelimb_He_E14', 'ChondroOsteo_Long', 'Suture2021_Farmer_E15', 'Frontal_Holmes_WTE18.5_1', 'Mesenchymal2022_Zhang_E11.5', 'Forelimb_He_E13_1', 'Frontal_Holmes_WTE18.5_4', 'Frontal_Holmes_PN10_Posterior', 'CranioSoxc_Angelozzi_WTE13.5', 'coronal_Hol

In [44]:
cpmDict_mes=mdata["milo"].uns["cpm_dict"]

In [45]:
# Specify the directory where you want to save the CSV files
output_directory = '../processed_data/12.12_TRAV/cpmList/mesenchyme/'

# Export each DataFrame in the dictionary to a separate CSV file
for df_name, df in cpmDict_mes.items():
    df.to_csv(output_directory + f'{df_name}.csv', index=True)

In [46]:
dpt_lepr=dpt[dpt.obs["lineage_lepr"]]
mdata_lepr = milo2.load(dpt_lepr)
mdata_lepr=runCPM(mdata_lepr)

/home/zhanglab/mambaforge/envs/celloracle/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
100%|██████████| 934/934 [09:54<00:00,  1.57it/s]


Using edgeR to find CPM......

  0%|          | 1/934 [00:28<7:30:43, 28.99s/it]Unable to determine R library path: Command '('/usr/lib/R/bin/Rscript', '-e', 'cat(Sys.getenv("LD_LIBRARY_PATH"))')' returned non-zero exit status 2.
Unable to determine R library path: Command '('/usr/lib/R/bin/Rscript', '-e', 'cat(Sys.getenv("LD_LIBRARY_PATH"))')' returned non-zero exit status 2.
Unable to determine R library path: Command '('/usr/lib/R/bin/Rscript', '-e', 'cat(Sys.getenv("LD_LIBRARY_PATH"))')' returned non-zero exit status 2.
Unable to determine R library path: Command '('/usr/lib/R/bin/Rscript', '-e', 'cat(Sys.getenv("LD_LIBRARY_PATH"))')' returned non-zero exit status 2.
Unable to determine R library path: Command '('/usr/lib/R/bin/Rscript', '-e', 'cat(Sys.getenv("LD_LIBRARY_PATH"))')' returned non-zero exit status 2.
Unable to determine R library path: Command '('/usr/lib/R/bin/Rscript', '-e', 'cat(Sys.getenv("LD_LIBRARY_PATH"))')' returned non-zero exit status 2.
Unable to determine R library path: Command '('/usr

In [51]:
cpm_lepr=mdata_lepr['milo'].uns["cpm_dict"]

In [52]:
output_directory = '../processed_data/12.12_TRAV/cpmList/lepr/'
for df_name, df in cpm_lepr.items():
    df.to_csv(output_directory + f'{df_name}.csv', index=True)

In [47]:
dpt_fibro=dpt[dpt.obs["lineage_laFibro"]]
mdata_fibro = milo2.load(dpt_fibro)
mdata_fibro=runCPM(mdata_fibro)

/home/zhanglab/mambaforge/envs/celloracle/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
100%|██████████| 369/369 [03:15<00:00,  1.89it/s]


Using edgeR to find CPM......

  0%|          | 1/369 [00:29<3:01:07, 29.53s/it]Unable to determine R library path: Command '('/usr/lib/R/bin/Rscript', '-e', 'cat(Sys.getenv("LD_LIBRARY_PATH"))')' returned non-zero exit status 2.
Unable to determine R library path: Command '('/usr/lib/R/bin/Rscript', '-e', 'cat(Sys.getenv("LD_LIBRARY_PATH"))')' returned non-zero exit status 2.
Unable to determine R library path: Command '('/usr/lib/R/bin/Rscript', '-e', 'cat(Sys.getenv("LD_LIBRARY_PATH"))')' returned non-zero exit status 2.
Unable to determine R library path: Command '('/usr/lib/R/bin/Rscript', '-e', 'cat(Sys.getenv("LD_LIBRARY_PATH"))')' returned non-zero exit status 2.
Unable to determine R library path: Command '('/usr/lib/R/bin/Rscript', '-e', 'cat(Sys.getenv("LD_LIBRARY_PATH"))')' returned non-zero exit status 2.
Unable to determine R library path: Command '('/usr/lib/R/bin/Rscript', '-e', 'cat(Sys.getenv("LD_LIBRARY_PATH"))')' returned non-zero exit status 2.
Unable to determine R library path: Command '('/usr

In [54]:
output_directory = '../processed_data/12.12_TRAV/cpmList/fibro/'
cpm_fibro=mdata_fibro['milo'].uns["cpm_dict"]
for df_name, df in cpm_fibro.items():
    df.to_csv(output_directory + f'{df_name}.csv', index=True)

In [48]:
dpt_chondro=dpt[dpt.obs["lineage_chondro"]]
mdata_chondro = milo2.load(dpt_chondro)
mdata_chondro=runCPM(mdata_chondro)

/home/zhanglab/mambaforge/envs/celloracle/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
100%|██████████| 300/300 [02:45<00:00,  1.81it/s]


Using edgeR to find CPM......

  0%|          | 1/300 [00:29<2:25:57, 29.29s/it]Unable to determine R library path: Command '('/usr/lib/R/bin/Rscript', '-e', 'cat(Sys.getenv("LD_LIBRARY_PATH"))')' returned non-zero exit status 2.
Unable to determine R library path: Command '('/usr/lib/R/bin/Rscript', '-e', 'cat(Sys.getenv("LD_LIBRARY_PATH"))')' returned non-zero exit status 2.
Unable to determine R library path: Command '('/usr/lib/R/bin/Rscript', '-e', 'cat(Sys.getenv("LD_LIBRARY_PATH"))')' returned non-zero exit status 2.
Unable to determine R library path: Command '('/usr/lib/R/bin/Rscript', '-e', 'cat(Sys.getenv("LD_LIBRARY_PATH"))')' returned non-zero exit status 2.
Unable to determine R library path: Command '('/usr/lib/R/bin/Rscript', '-e', 'cat(Sys.getenv("LD_LIBRARY_PATH"))')' returned non-zero exit status 2.
Unable to determine R library path: Command '('/usr/lib/R/bin/Rscript', '-e', 'cat(Sys.getenv("LD_LIBRARY_PATH"))')' returned non-zero exit status 2.
Unable to determine R library path: Command '('/usr

In [55]:
output_directory = '../processed_data/12.12_TRAV/cpmList/chondro/'
cpm_chondro=mdata_chondro['milo'].uns["cpm_dict"]
for df_name, df in cpm_chondro.items():
    df.to_csv(output_directory + f'{df_name}.csv', index=True)

In [ ]:
mdata=mdata_mes
sc.pp.neighbors(mdata['rna'], use_rep='X_scANVI', n_neighbors=200, n_pcs=15)
milo2.make_nhoods(mdata['rna'], prop=0.015)
mdata = milo2.count_nhoods(mdata['rna'], sample_col="Sample")
pseudobulk=milo2.make_pseudobulk_parallel(mdata=mdata,sample_col="Sample",group_col="Stage",time_col="light_pred",njob=25,min_cell=2)
milo2.make_range(mdata,only_range=True)
wholeCpm=milo2.make_whole_cpm(mdata)
milo2.permute_point_cpm_parallel(mdata)